In [1]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''
    CREATE TABLE IF NOT EXISTS records (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        sitename TEXT NOT NULL,
        county TEXT,
        aqi INTEGER,
        status TEXT,
        pm25 NUMERIC,
        date TEXT,
        lat NUMERIC,
        lon NUMERIC,
        UNIQUE(sitename,date)
    );
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [2]:
import requests

url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    for items in data['records']:
        sitenames.add(items['sitename'])

    sitenames = list(sitenames)
print(sitenames)

['淡水', '新北(樹林)', '臺南（麻豆）', '竹山', '仁武', '關山', '新莊', '彰化（員林）', '宜蘭（頭城）', '復興', '龍潭', '大園', '埔里', '沙鹿', '楠梓', '線西', '竹東', '士林', '屏東', '屏東(枋山)', '基隆', '潮州', '萬里', '崙背', '板橋', '林口', '前金', '南投', '土城', '西屯', '三重', '美濃', '中山', '陽明', '大寮', '三義', '宜蘭', '馬祖', '二林', '臺東', '屏東（琉球）', '小港', '永和', '平鎮', '觀音', '馬公', '菜寮', '富貴角', '桃園', '臺西', '前鎮', '林園', '花蓮', '大同', '新港', '大城', '恆春', '安南', '斗六', '金門', '古亭', '萬華', '臺南', '冬山', '善化', '鳳山', '松山', '嘉義', '大里', '橋頭', '中壢', '頭份', '麥寮', '新竹', '豐原', '忠明', '彰化', '苗栗', '高雄（湖內）', '左營', '汐止', '新營', '朴子', '湖口', '新店']


In [3]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
    values ('屏東(枋山)','屏東縣', 34, '良好',18,'2024-11-04 10:00', 22.260899, 120.651472);
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [4]:
import sqlite3
conn = sqlite3.connect("AQI.db")
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    with conn:
        cursor = conn.cursor()
        for items in data['records']:
            sitename = items['sitename']
            county = items['county']
            aqi = int(items['aqi']) if items['aqi'] != '' else 0
            status = items['status']
            pm25 = float(items['pm2.5']) if items['pm2.5'] != '' else 0.0
            date = items['datacreationdate']
            lon = float(items['longitude']) if items['longitude'] != '' else 0.0
            lat = float(items['latitude']) if items['latitude'] != '' else 0.0
            sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
                    values (?,?, ?, ?,?,?,?,?);
            '''
            cursor.execute(sql,(sitename, county, aqi, status,pm25,date,lat,lon))


In [6]:
conn = sqlite3.connect("AQI.db")
with conn:
    cursor = conn.cursor()
    sql = '''
    SELECT DISTINCT sitename
    FROM records
    '''
    cursor.execute(sql)
    sitenames = []

    for items in cursor.fetchall():
        sitenames.append(items[0])
    
    print(sitenames)

['三義', '三重', '中壢', '中山', '二林', '仁武', '冬山', '前金', '前鎮', '南投', '古亭', '善化', '嘉義', '土城', '埔里', '基隆', '士林', '大同', '大園', '大城', '大寮', '大里', '安南', '宜蘭', '宜蘭（頭城）', '富貴角', '小港', '屏東', '屏東(枋山)', '屏東（琉球）', '崙背', '左營', '平鎮', '彰化', '彰化（員林）', '復興', '忠明', '恆春', '斗六', '新北(樹林)', '新店', '新港', '新營', '新竹', '新莊', '朴子', '松山', '板橋', '林口', '林園', '桃園', '楠梓', '橋頭', '永和', '汐止', '沙鹿', '淡水', '湖口', '潮州', '竹山', '竹東', '線西', '美濃', '臺南', '臺南（麻豆）', '臺東', '臺西', '花蓮', '苗栗', '菜寮', '萬華', '萬里', '西屯', '觀音', '豐原', '金門', '關山', '陽明', '頭份', '馬公', '馬祖', '高雄（湖內）', '鳳山', '麥寮', '龍潭']


In [ ]:
conn = sqlite3.connect('AQI.db')
with conn:
    cursor = conn.cursor()
    sql = '''
    SELECT  date,county,aqi,pm2.5,status,lat,lon
    form records
    WHERE sitname = ?
    ORDER  BY  date DESC;
    '''
    cursor.execute(sql,('富貴角'))
    sitename_list = list